## Notebook to test TherAPPy

The objective of this notebook is to show the functionality of TherAPPy

Note that the code is being developed and the structure and fucntionality of the code will change and improve. Get in touch with the developers if you feel that anything could be improved in particular. 

The code currently includes:

- the Dodson (1973) cooling age model for a range of thermochronometers
- The Ketcham (1999, 2007) apatite fission track annealing model



In [1]:
from importlib import reload
import itertools
import numpy as np
import matplotlib.pyplot as pl
import matplotlib.colors
import astropy.units as u

import therappy.therappy_lib as trp

ModuleNotFoundError: No module named 'lib.therappy_lib'

## Set up a temperature history

The temperature history is stored in a python dictionary that contains two numpy arrays, one for time bp and one for temperature.
Note that the unit for time used in TherAPPy is years bp, and the unit for temperature is degrees C. TherAPPy uses the [astropy units](https://docs.astropy.org/en/stable/units/index.html) module to assign physical units to variables. 

In [ ]:
time = np.arange(0, 100.0, 1.0)[::-1] * 1e6 * u.year
temperature =  np.linspace(300, 10, 100)  * u.deg_C
tT = {"time": time, "temperature": temperature}

print(f"Temperature history:\n{tT}")

## Model closure ages using the Dodson model

We first model the closure ages following Dodson (1973) for the apatite and zircon (U-Th)/He and fission track thermochronometers, using parameters from Reiners and Brandon (2006, https://doi.org/10.1146/annurev.earth.34.031405.125202)

In [ ]:
minerals = ["apatite", "apatite", "zircon", "zircon"]
thermochronometers = ["He", "FT", "He", "FT"]

grains = []
ages = []

# go through the thermochron systems
for mineral, thermochronometer in zip(minerals, thermochronometers):
    
    # create a new thermochron object for each mineral & thermochronometer combination
    mygrain = trp.thermochron_object(mineral) # geometry (size, shape), chemistry (eU / U / Th, Sm), zonation?
    
    # todo
    #mysample = sample(grain1, grain2)
    
    # model the thermochron age
    model_results = mygrain.model_thermochron(tT, thermochronometer=thermochronometer, model="Dodson") #dpar, 
    
    # store the ages
    ages.append(model_results["modelled_thermochron_age_bp"])
    
    # store the grain in a list
    grains.append(mygrain)
    
    print(f"modelled thermochron age for the system {mygrain.mineral} {thermochronometer} using the {mygrain.model} model: "
          f"{model_results['modelled_thermochron_age_bp']:0.2e}")

## Model AFT using the Ketcham (2007) model

Next we model the apatite fission track age using a implementation of the fission track annealing model described in Ketcham et al. (1999) and Ketcham (2007). The annealing resistance can be specified using the chlorine weight fraction or the Dpar parameter. We assing a chlorine weight fraction of 0.0 below.

In [ ]:
# assign fission track annealing parameters
thermochron_parameters = {"kinetic_parameter": "Clwt", "Clwt": 0.0}

# set up a new grain
FTgrain = trp.thermochron_object("apatite")

# model the apatite fission track age
model_results = FTgrain.model_thermochron(tT, thermochronometer="FT", model="Ketcham2007", thermochron_parameters=thermochron_parameters)

modelled_AFT_age = model_results["AFT_age"]

print(f"modelled apatite fission track age: {modelled_AFT_age :0.2f}")

## Show T history and modelled ages in a figure

Lets compare the closure ages with the modelled apatite fission track age:

In [ ]:
colors = list(matplotlib.colors.TABLEAU_COLORS)

fig, ax = pl.subplots(1, 1, figsize=(6, 4))
ax.plot(tT["time"].value / 1e6, tT["temperature"], label="T history")
ax.set_ylim(tT["temperature"].value.max(), 0)
ax.set_xlabel("Time (My bp)")
Tu = tT["temperature"].unit
ax.set_ylabel(f"Temperature ({Tu})")
ax.set_title("Modelled temperature history\nand thermochronometer ages")

for i, grain, age in zip(itertools.count(), grains, ages):
    mineral = grain.mineral
    thermochronometer = grain.thermochronometer
    ax.axvline(age.value / 1e6, c=colors[i+1], ls=":", lw=2.0, label=f"{mineral} {thermochronometer} age,\n Dodson (1973)")

ax.axvline(modelled_AFT_age.value / 1e6, c=colors[i+2], ls="--", lw=1.5, label="AFT age,\n Ketcham (2007)")

ax.legend(bbox_to_anchor=(1, 0.75))

ax.set_xlim(tT["time"].value.max() / 1e6, 0)

fig.tight_layout()

fig.savefig("fig/TherAPPy_example.png")

## Show change in AFT ages over time

In [ ]:
age_evolution = model_results["AFT_ages"]

fig, ax = pl.subplots(1, 1, figsize=(6, 4))

x, y = tT["time"].value / 1e6, tT["temperature"].value
xm = x[int(len(x) / 2)]
ym = y[int(len(y) / 2)]
ax.plot(x, y, color=colors[0], label="T history")
ax.text(xm, ym, "temperature", color=colors[0], va="center", rotation=35)

axr = ax.twinx()

x = time[1:].value / 1e6
y = age_evolution.value / 1e6
xm = x[int(len(x) / 2)]
ym = y[int(len(y) / 2)]
axr.plot(x, y, color=colors[1])
axr.text(xm, ym, "AFT age", color=colors[1], va="bottom")

ax.set_xlabel("Time (My bp)")
ax.set_ylabel(f"Temperature ({Tu})")
axr.set_ylabel("AFT age (My bp)")

ax.set_xlim(tT["time"].value.max() / 1e6, 0)
ax.set_ylim(tT["temperature"].value.max(), 0)

ax.set_title("Modelled temperature history\nand AFT age over time")